In [44]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta
import matplotlib.pyplot as plt

import datetime

plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('fivethirtyeight')

In [58]:
tickers = ['F']
start = '2015-01-01'
end = '2020-01-01'
trackpoint = datetime.datetime.strptime(end, '%Y-%m-%d') - datetime.timedelta(days=10*365)

stock = yf.download(tickers, start, end)
stock

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,15.59,15.65,15.18,15.36,11.553761,24777900
2015-01-05,15.12,15.13,14.69,14.76,11.102440,44079700
2015-01-06,14.88,14.90,14.38,14.62,10.997134,32981600
2015-01-07,14.78,15.09,14.77,15.04,11.313056,26065300
2015-01-08,15.40,15.48,15.23,15.42,11.598891,33943400
...,...,...,...,...,...,...
2019-12-24,9.44,9.49,9.43,9.47,9.217597,11881600
2019-12-26,9.47,9.49,9.43,9.45,9.198129,28961300
2019-12-27,9.45,9.46,9.35,9.36,9.110528,28272800


In [59]:
rsi_days = 28
lower_bar = 40
upper_bar = 70
RSI_data = ta.RSI(stock['Adj Close'], rsi_days)
stock_data = pd.DataFrame(stock['Adj Close'], columns=['Adj Close'])


buying_signal = pd.DataFrame(columns=['Adj Close'])
selling_signal = pd.DataFrame(columns=['Adj Close'])
equity = 0

In [60]:

for x in range(rsi_days, len(stock_data)):

    #Buying
    if RSI_data[x-rsi_days]<lower_bar and equity<5:
        buying_signal = buying_signal.append(stock_data.iloc[x])
        equity += 1

    # Selling
    elif RSI_data[x-rsi_days]>upper_bar and equity>0:
        for y in range(0,equity):
            selling_signal = selling_signal.append(stock_data.iloc[x])
            equity -= 1

In [61]:
def calc_profits(ret, ss, bs, sd):
    if len(ss)!=0 and len(bs)!=0:
        ret *= (ss[0]/bs[0])*calc_profits(ret, ss[1:], bs[1:], sd)
    elif len(ss)==0 and len(bs)!=0:
        ret *= (sd[-1]/bs[0])*calc_profits(ret, ss[1:], bs[1:], sd)
        
    return ret

In [62]:
ss = selling_signal['Adj Close']
bs = buying_signal['Adj Close']
sd = stock_data['Adj Close']
'''
if len(bs)>len(ss):
    returns = np.prod(ss[:]/bs[:len(bs)-len(ss)]) * np.prod((sd[-1])**(len(bs)-len(ss))/bs[len(bs)-len(ss):])

else:
    returns = np.prod(ss/bs)
'''

returns = calc_profits(1, ss, bs, sd)

print("Returns: ", returns)
print("Equity: ", equity)
print("\nBuying Signals: \n", bs)
print("\nSelling Signals: \n",ss)

Returns:  0.7696188372037313
Equity:  5

Buying Signals: 
 2015-07-16    11.175736
2015-08-17    11.375166
2015-08-18    11.491398
2015-08-19    11.444905
2015-10-01    10.592545
2018-03-07     9.274724
2018-03-08     9.257273
2018-03-09     9.361973
2018-03-12     9.431774
2018-03-13     9.405599
2019-09-24     8.724018
2019-09-25     8.810205
2019-09-26     8.752748
2019-09-30     8.771899
2019-10-03     8.340965
Name: Adj Close, dtype: float64

Selling Signals: 
 2017-11-10    10.361848
2017-11-10    10.361848
2017-11-10    10.361848
2017-11-10    10.361848
2017-11-10    10.361848
2019-06-06     9.199595
2019-06-06     9.199595
2019-06-06     9.199595
2019-06-06     9.199595
2019-06-06     9.199595
Name: Adj Close, dtype: float64
